In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tables
import mplfinance as mpf
import my_afml_fncs as my_afml

In [3]:
volume_bars_df = pd.read_hdf("tick_data/bars/volume_bars.h5", key="volume_bars")
volume_bars_df.head()

,time,open,high,low,close,volume
0,2024-11-15 08:39:56.469400,5967.00,5968.00,5930.75,5937.00,100001
1,2024-11-15 13:33:55.913324,5937.25,5953.25,5936.25,5944.25,100003
2,2024-11-15 14:30:17.843538,5944.50,5947.75,5935.00,5936.75,100040
3,2024-11-15 14:43:36.170628,5936.75,5941.75,5929.25,5930.00,100037
4,2024-11-15 15:02:28.997004,5929.75,5940.50,5920.00,5920.25,100002


In [4]:
volume_bars_df['time']=pd.to_datetime(volume_bars_df['time'])
volume_bars_df.set_index('time',inplace=True)
vol=my_afml.get_daily_volatility(volume_bars_df.close)

In [5]:
vol2=my_afml.getDailyVol(volume_bars_df.close)
vol2.mean()

0.0037081056391128505

In [6]:
vol2.mean()
vol.mean()

0.0037081056391128505

In [7]:
es_cusum_events=my_afml.getTEvents(volume_bars_df.close,vol2.mean())

In [8]:

t1=my_afml.getVb(volume_bars_df.close,es_cusum_events)
t1

2024-11-15 13:33:55.913324   2024-11-18 06:58:08.108243
2024-11-15 14:30:17.843538   2024-11-18 06:58:08.108243
2024-11-15 14:43:36.170628   2024-11-18 06:58:08.108243
2024-11-15 15:02:28.997004   2024-11-18 06:58:08.108243
2024-11-15 15:16:29.237773   2024-11-18 06:58:08.108243
                                        ...            
2024-12-16 20:59:59.504389   2024-12-18 14:34:03.919800
2024-12-17 14:38:13.296365   2024-12-18 19:08:29.806640
2024-12-17 16:31:24.270556   2024-12-18 19:08:29.806640
2024-12-17 20:45:34.771591   2024-12-18 21:03:19.567693
2024-12-18 14:34:03.919800   2024-12-19 17:26:20.544849
Name: time, Length: 257, dtype: datetime64[ns]

In [9]:
ptsl=1
trgt=vol2
minRet=0.00003
numThread=1
trigger=my_afml.getEvents(volume_bars_df.close,es_cusum_events,ptsl,trgt,minRet,numThread,t1)
print(trigger)
print(trigger.shape)

Running processJobs_
                                                   t1      trgt
2024-11-18 13:00:00.991865 2024-11-18 15:29:54.497644  0.001768
2024-11-18 14:35:36.212295 2024-11-18 15:29:54.497644  0.001770
2024-11-18 15:00:41.163755 2024-11-18 15:29:54.497644  0.001564
2024-11-18 15:29:54.497644 2024-11-18 16:46:11.699835  0.001548
2024-11-18 16:00:36.713567 2024-11-19 09:28:48.026253  0.001741
...                                               ...       ...
2024-12-18 19:08:29.806640 2024-12-18 20:33:00.914784  0.003980
2024-12-18 20:33:00.914784 2024-12-18 21:03:19.567693  0.004736
2024-12-18 21:03:19.567693 2024-12-19 14:32:08.428649  0.006430
2024-12-19 14:32:08.428649                        NaT  0.006900
2024-12-19 17:26:20.544849                        NaT  0.007644

[244 rows x 2 columns]
(244, 2)


In [10]:
bins=my_afml.getBins(trigger,volume_bars_df.close)
print(bins)
print(bins.shape)

                                 ret  bin
2024-11-18 13:00:00.991865  0.001864  1.0
2024-11-18 14:35:36.212295  0.002799  1.0
2024-11-18 15:00:41.163755  0.002586  1.0
2024-11-18 15:29:54.497644  0.002918  1.0
2024-11-18 16:00:36.713567 -0.004942 -1.0
...                              ...  ...
2024-12-17 20:45:34.771591 -0.004295 -1.0
2024-12-18 14:34:03.919800 -0.018100 -1.0
2024-12-18 19:08:29.806640 -0.014516 -1.0
2024-12-18 20:33:00.914784 -0.013720 -1.0
2024-12-18 21:03:19.567693  0.012119  1.0

[242 rows x 2 columns]
